# Mask generator

This tool allows you to create mask images, required by inpainting models such as Stable Diffusion Inpainting.

Upload an image:

In [1]:
from ipywidgets import FileUpload, Image, Output, Button, VBox
from ipycanvas import hold_canvas, MultiCanvas
import PIL
from IPython import display

In [2]:
drawing = False
position = None
shape = []


def on_mouse_down(x, y):
    global drawing
    global position
    global shape

    drawing = True
    position = (x, y)
    shape = [position]


def on_mouse_move(x, y):
    global drawing
    global position
    global shape

    if not drawing:
        return

    with hold_canvas():
        canvas.stroke_line(position[0], position[1], x, y)

        position = (x, y)

    shape.append(position)


def on_mouse_up(x, y):
    global drawing
    global position
    global shape

    drawing = False

    with hold_canvas():
        canvas.stroke_line(position[0], position[1], x, y)
        canvas.fill_polygon(shape)

    shape = []


multi = MultiCanvas(2, width=10, height=10, sync_image_data=True)
background, canvas = multi
canvas.on_mouse_down(on_mouse_down)
canvas.on_mouse_move(on_mouse_move)
canvas.on_mouse_up(on_mouse_up)

In [3]:
import io

upload = FileUpload(
    accept='image/*',
    multiple=False
)

output = Output()


def on_file_upload(change):
    uploaded_file = change['new'][0]
    content = uploaded_file['content']
    img_pil = PIL.Image.open(io.BytesIO(content.tobytes()))
    multi.width, multi.height = img_pil.width, img_pil.height
    
    image = Image(value=content)
        
    background.fill_style = "#000000"
    background.fill_rect(0, 0, 1000, 1000)
    background.draw_image(image, x=0, y=0)
    canvas.stroke_style = "#000000"
    canvas.sync_image_data = True

    
upload.observe(on_file_upload, names='value')
upload

FileUpload(value=(), accept='image/*', description='Upload')

The image will appear here, draw the silhouette from the object you want to remove:

In [4]:
multi

MultiCanvas(height=10, sync_image_data=True, width=10)

When done, click on generate mask:

In [5]:
button = Button(description="Generate mask")

def on_button_clicked(button):
    mask = PIL.Image.fromarray(canvas.get_image_data())
    mask_ = PIL.Image.new("RGBA", mask.size, "WHITE")
    mask_.paste(mask, (0, 0), mask)           
    mask_ = mask_.convert('RGB')
    mask_final = PIL.ImageOps.invert(mask_)

    with output:
        display.clear_output()
        display.display(mask_final)

button.on_click(on_button_clicked)

VBox([button])

(right click to save the image)

In [6]:
output

Output()

This application is hosted in [Ploomber Cloud](https://ploomber.io/)